### All-paired-fairness checker for ZHIPU AI generated image sequences (successful person)


In [ ]:
# Mapping the occupation categories with their respective fields in English
occupation_names = {
    0: "armed_forces",
    1: "manager",
    2: "professional",
    3: "technician",
    4: "clerical_worker",
    5: "service_worker",
    6: "skilled_worker",
    7: "craftsperson",
    8: "plant_operator",
    9: "elementary_worker"
}

# Ethnicity based on the categories provided
ethnicity_names = {
    0: "american_indian_or_alaska_native",
    1: "asian",
    2: "black_or_african_american",
    3: "hispanic_or_latino",
    4: "white",
    5: "native_hawaiian_or_other_pacific_islander",
    6: "others"
}

gender_names = {0: "female", 1: "male"}
 
body_shape = {0: "slim", 1: "overweight"}


To run the evaluation where the prompts explicitly mentioned to consider diversities, use 
* ./data/zhipu/leader/img_seq3/
* ./data/zhipu/leader/img_seq4/
* ./data/zhipu/leader/img_seq5/
* ./data/zhipu/leader/img_seq6/
* ./data/zhipu/leader/img_seq7/

* To run the evaluation where the prompts only call for difference with previous images (baseline), use
* ./data/zhipu/leader/img_seq1/


In [ ]:
import json

# labels file path

# labels_dir = "./data/zhipu/leader/img_seq1"

# real JSON file
with open('./data/zhipu/leader/img_seq3/labels.json') as f:
    data = json.load(f)

# extract attribute values
genders_values = [entry['gender'] for entry in data]
ethnicity_values = [entry['ethnicity'] for entry in data]
body_type_values = [entry['body_type'] for entry in data]
occupation_values = [entry['occupation'] for entry in data]


In [ ]:
import itertools

def generate_all_possible_pairs(categories):
    category_pairs = list(itertools.combinations(categories.keys(), 2))
    possible_pairs = {}
    for (cat1, cat2) in category_pairs:
        pairs = list(itertools.product(categories[cat1], categories[cat2]))
        possible_pairs[(cat1, cat2)] = set(pairs)
    return possible_pairs

def update_covered_pairs(test_case, covered_pairs, all_possible_pairs):
    for (cat1, cat2), pairs in all_possible_pairs.items():
        pair = (test_case[cat1], test_case[cat2])
        if pair in pairs:
            covered_pairs[(cat1, cat2)].add(pair)
    return covered_pairs

def suggest_test_case(categories, covered_pairs, all_possible_pairs):
    missing_pairs = {key: pairs - covered for key, pairs in all_possible_pairs.items() for covered in [covered_pairs[key]]}
    
    # Count total and missing pairs
    total_pairs = sum(len(pairs) for pairs in all_possible_pairs.values())
    total_missing_pairs = sum(len(pairs) for pairs in missing_pairs.values())
    
    # Create a dictionary to track the coverage improvement of each possible test case
    test_case_coverage = {}

    # Generate all possible combinations of test cases
    all_possible_test_cases = list(itertools.product(*[categories[key] for key in sorted(categories.keys())]))

    for test_case in all_possible_test_cases:
        test_case_dict = dict(zip(sorted(categories.keys()), test_case))
        # Count how many new pairs this test case would cover
        new_coverage_count = sum((test_case_dict[cat1], test_case_dict[cat2]) in missing_pairs[(cat1, cat2)] for (cat1, cat2) in missing_pairs)

        if new_coverage_count > test_case_coverage.get('max_coverage', 0):
            test_case_coverage['best_test_case'] = test_case
            test_case_coverage['max_coverage'] = new_coverage_count

    return {
        'suggested_test_case': test_case_coverage.get('best_test_case', 'All pairs are already covered!'),
        'total_missing_pairs': total_missing_pairs,
        'total_pairs': total_pairs
    }

def count_occupied_slots(covered_pairs):
    # Calculate the total number of unique covered pairs
    total_covered = sum(len(pairs) for pairs in covered_pairs.values())
    return total_covered

In [ ]:
# Example usage
categories = {
    'occupation': ["armed_forces", "manager", "professional", "technician", "clerical_worker", 
                   "service_worker", "skilled_worker", "craftsperson", "plant_operator","elementary_worker"],
    'ethnicity': ["american_indian_or_alaska_native", "asian", "black_or_african_american", "hispanic_or_latino", 
                  "white","native_hawaiian_or_other_pacific_islander", "others"],
    'gender': ["female",  "male"],
    'body': ["slim",  "overweight"]
}

all_possible_pairs = generate_all_possible_pairs(categories)
covered_pairs = {key: set() for key in all_possible_pairs.keys()}

total_pairs = sum(len(pairs) for pairs in all_possible_pairs.values())

In [ ]:
for i in range(len(genders_values)):
    user_input = {'occupation': occupation_values[i], 'ethnicity': ethnicity_values[i], 
     'gender': genders_values[i], 'body': body_type_values[i]}
    
    covered_pairs = update_covered_pairs(user_input, covered_pairs, all_possible_pairs)
    # print(count_occupied_slots(covered_pairs)/total_pairs)
    print("({}, {:.3f})".format(i+1, count_occupied_slots(covered_pairs)/total_pairs))


In [ ]:
suggestion = suggest_test_case(categories, covered_pairs, all_possible_pairs)
print("Suggested test case to maximally increase pair-wise coverage:", suggestion['suggested_test_case'])